데이터 전처리는 저번주 수요일에 이야기했던 것처럼 간단하게 stopwords 제거하고 punctuation 같은 것만 정리한 뒤에
tensorflow에 있는 learn.preprocessing.VocabularyProcessor를 이용하여 embedding해 주었습니다.
(토요일에 조언받았을 때 pretrain된 것 쓰는 것보다 그냥 embedding하는 게 더 나을 것 같다고 말씀해주셨습니다.
 그리고 multi-label의 경우에도 저번에 말했던 것처럼 그냥 각각에 대해서 sigmoid 함수를 취하면 된 것 같다고 해 주셨습니다.)
 all-zero label의 경우에는 새로운 column을 하나 추가해서 모두 0이면 1의 값을 주는 식으로 학습을 시킨다면 성능이 향상될 수 있다고 하셔서
 일단 두 가지 모두 동일한 조건으로 시도해봤습니다.)

In [55]:
import pandas as pd

train = pd.read_csv("train.csv")
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [56]:
test = pd.read_csv("test.csv")
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [57]:
import nltk
import re

stop_words = set(nltk.corpus.stopwords.words('english'))

def preprocess_input(t):
    t = t.strip()
    z = re.findall(r'[A-Za-z]+', t)
    z = [a for a in z if len(a) > 2]
    wnlemma = nltk.stem.WordNetLemmatizer()
    z = [wnlemma.lemmatize(a) for a in z]
    z = [a for a in z if not a in stop_words]
    t = ' '.join(z)
    return t

In [58]:
train_cleaned = train

In [59]:
train_cleaned.comment_text = train.comment_text.apply(lambda x: preprocess_input(x))

In [60]:
train_cleaned.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation Why edits made username Hardcore M...,0,0,0,0,0,0
1,000103f0d9cfb60f,aww match background colour seemingly stuck Th...,0,0,0,0,0,0
2,000113f07ec002fd,Hey man really trying edit war guy constantly ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,More make real suggestion improvement wondered...,0,0,0,0,0,0
4,0001d958c54c6e35,You sir hero Any chance remember page,0,0,0,0,0,0


In [61]:
train_cleaned.to_csv('preprocessed.csv',index=False)

In [57]:
cols_target = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
train_target = train_cleaned[cols_target]
train_target.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [62]:
test_cleaned = test

In [63]:
test_cleaned.comment_text = test.comment_text.apply(lambda x: preprocess_input(x))

In [64]:
test_cleaned.head()

,id,comment_text
0,00001cee341fdb12,bitch Rule succesful ever whats hating sad mof...
1,0000247867823ef7,From RfC The title fine IMO
2,00013b17ad220c46,Sources Zawe Ashton Lapland
3,00017563c3f7919a,look back source information updated wa correc...
4,00017695ad8997eb,anonymously edit article


In [65]:
test_cleaned.to_csv('preprocessed_test.csv',index=False)

In [1]:
import pandas as pd

train_cleaned = pd.read_csv('preprocessed.csv')
train_cleaned.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation Why edits made username Hardcore M...,0,0,0,0,0,0
1,000103f0d9cfb60f,aww match background colour seemingly stuck Th...,0,0,0,0,0,0
2,000113f07ec002fd,Hey man really trying edit war guy constantly ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,More make real suggestion improvement wondered...,0,0,0,0,0,0
4,0001d958c54c6e35,You sir hero Any chance remember page,0,0,0,0,0,0


In [40]:
#add 7th category to target
az=np.zeros(len(train_cleaned), dtype=np.int64)
az.shape

(159571,)

In [55]:
target=train_cleaned[cols_target].values
target.shape

(159571, 6)

In [41]:
for i in range(len(train_cleaned)):
    if(sum(target[i])==0):
        az[i]=1

In [43]:
train_cleaned_7 = pd.read_csv('preprocessed.csv')
train_cleaned_7['all_zero']=az
train_cleaned_7.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,all_zero
0,0000997932d777bf,Explanation Why edits made username Hardcore M...,0,0,0,0,0,0,1
1,000103f0d9cfb60f,aww match background colour seemingly stuck Th...,0,0,0,0,0,0,1
2,000113f07ec002fd,Hey man really trying edit war guy constantly ...,0,0,0,0,0,0,1
3,0001b41b1c6bb37e,More make real suggestion improvement wondered...,0,0,0,0,0,0,1
4,0001d958c54c6e35,You sir hero Any chance remember page,0,0,0,0,0,0,1


In [49]:
train_cleaned_7.to_csv('preprocessed_7.csv', index=False)

In [2]:
test_cleaned = pd.read_csv('preprocessed_test.csv')
test_cleaned.head()

,id,comment_text
0,00001cee341fdb12,bitch Rule succesful ever whats hating sad mof...
1,0000247867823ef7,From RfC The title fine IMO
2,00013b17ad220c46,Sources Zawe Ashton Lapland
3,00017563c3f7919a,look back source information updated wa correc...
4,00017695ad8997eb,anonymously edit article


In [76]:
from tensorflow.contrib import learn
import numpy as np

vocab_processor = learn.preprocessing.VocabularyProcessor(500)
X = np.array(list(vocab_processor.fit_transform(train_cleaned['comment_text'].astype(str))))

In [77]:
X_test = np.array(list(vocab_processor.fit_transform(test_cleaned['comment_text'].astype(str))))

In [59]:
Y_train = train_target.values.astype('float32') 

In [71]:
az=az.reshape(159571,1)
az.shape

(159571, 1)

In [72]:
Y_train_7 = np.concatenate((Y_train,az), axis=1)
Y_train_7.shape

(159571, 7)

In [73]:
import tensorflow as tf
#import numpy as np


class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer 
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W ,b, name = "scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")
        
        with tf.name_scope("prob_each"):
            self.prob_each = tf.nn.sigmoid(self.scores, name="prob_each")
       
        with tf.name_scope("loss"):
            losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
    

In [74]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [79]:
#import tensorflow as tf
#import numpy as np
import os
import time
import datetime
#from tensorflow.contrib import learn

# Parameters
# ==================================================

# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 25, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 118, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 10, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")

tf.logging.set_verbosity(tf.logging.INFO)



# Data Preparation
# ==================================================

# Load data
#print("Loading data...")
#x_text, y = #data_helpers.load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file)

# Build vocabulary
#max_document_length = max([len(x.split(" ")) for x in x_text])
#vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
#x = np.array(list(vocab_processor.fit_transform(x_text)))

# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(Y_train)))
x_shuffled = X[shuffle_indices]
y_shuffled = Y_train[shuffle_indices]

# Split train/test set
# TODO: This is very crude, should use cross-validation
dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(Y_train)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

#del x, y, x_shuffled, y_shuffled

print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))

Vocabulary Size: 199560
Train/Dev split: 143614/15957


In [10]:
result=[]

In [82]:

# Training
# ==================================================

with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=X.shape[1],
            num_classes=Y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3) 
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

       
        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)        
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)
        scores_summary = tf.summary.tensor_summary("scores", cnn.scores)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary]) #, grad_summaries_merged
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        
        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy, scores, prob_each= sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy, cnn.scores, cnn.prob_each],
                feed_dict)
            
            if step % 100 == 0:
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}".format(time_str, step, loss))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy, scores = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy, cnn.scores],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}".format(time_str, step, loss))
            if writer:
                writer.add_summary(summaries, step)
                
        def test_step(x_batch):
           
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.dropout_keep_prob: 1.0
            }
            prob_each = sess.run(cnn.prob_each,feed_dict)
            result.append(prob_each)
        
       
       # Generate batches
        batches = batch_iter(list(zip(X,Y_train), FLAGS.batch_size, FLAGS.num_epochs)
        #batches = batch_iter(list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        print("Start training for each batch.")
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            #if current_step % FLAGS.evaluate_every == 0:
            #    print("\nEvaluation:")
            #    dev_step(x_dev, y_dev, writer=dev_summary_writer)
            #    print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                #print("Saved model checkpoint to {}\n".format(path))
        print("Training is over.")
       
        batches_test = batch_iter(X_test, FLAGS.batch_size, num_epochs=1, shuffle=False)
        for batch in batches_test:
            test_step(batch)
       
        print("The End!")

Writing to C:\Users\User\Desktop\SNU\Deepest\Toxic Comment Classification\runs\1517809384

Start training for each batch.


ValueError: Cannot feed value of shape (118, 6) for Tensor 'input_y:0', which has shape '(?, 7)'

In [80]:
result_7=[]

In [83]:

# Training
# ==================================================

with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=X.shape[1], 
            num_classes=Y_train_7.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3) 
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

       
        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)        
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)
        scores_summary = tf.summary.tensor_summary("scores", cnn.scores)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary]) #, grad_summaries_merged
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        
        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy, scores, prob_each= sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy, cnn.scores, cnn.prob_each],
                feed_dict)
            
            if step % 100 == 0:
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}".format(time_str, step, loss))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy, scores = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy, cnn.scores],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}".format(time_str, step, loss))
            if writer:
                writer.add_summary(summaries, step)
                
        def test_step(x_batch):
           
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.dropout_keep_prob: 1.0
            }
            prob_each = sess.run(cnn.prob_each,feed_dict)
            result_7.append(prob_each)
        
       
       # Generate batches
        batches = batch_iter(list(zip(X,Y_train_7)), FLAGS.batch_size, FLAGS.num_epochs)
        print("Start training for each batch.")
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            #if current_step % FLAGS.evaluate_every == 0:
            #    print("\nEvaluation:")
            #    dev_step(x_dev, y_dev, writer=dev_summary_writer)
            #    print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                #print("Saved model checkpoint to {}\n".format(path))
        print("Training is over.")
       
        batches_test = batch_iter(X_test, FLAGS.batch_size, num_epochs=1, shuffle=False)
        for batch in batches_test:
            test_step(batch)
       
        print("The End!")

Writing to C:\Users\User\Desktop\SNU\Deepest\Toxic Comment Classification\runs\1517809492

Start training for each batch.
2018-02-05T14:47:34.880643: step 100, loss 0.196612
2018-02-05T14:50:11.907054: step 200, loss 0.149705
2018-02-05T14:52:47.140660: step 300, loss 0.172511
2018-02-05T14:55:24.508782: step 400, loss 0.140023
2018-02-05T14:58:04.214599: step 500, loss 0.124763
2018-02-05T15:00:41.720830: step 600, loss 0.147123
2018-02-05T15:03:16.821847: step 700, loss 0.0772475
2018-02-05T15:06:02.161157: step 800, loss 0.148117
2018-02-05T15:08:37.254665: step 900, loss 0.0565669
2018-02-05T15:11:19.435722: step 1000, loss 0.079805
2018-02-05T15:13:56.226943: step 1100, loss 0.134849
2018-02-05T15:16:33.858260: step 1200, loss 0.108149
2018-02-05T15:19:13.991353: step 1300, loss 0.153104
2018-02-05T15:22:03.612212: step 1400, loss 0.110159
2018-02-05T15:24:50.964455: step 1500, loss 0.0905614
2018-02-05T15:27:40.716409: step 1600, loss 0.0792628
2018-02-05T15:30:14.318359: step 17

In [12]:
import copy
df = copy.deepcopy(result)

In [14]:
result[0]

array([[  9.22572672e-01,   5.86731791e-01,   9.83838081e-01,
          2.74904538e-03,   3.73656652e-03,   5.59880900e-05],
       [  1.45068055e-03,   3.13559291e-03,   2.78067100e-03,
          3.43513791e-04,   3.55617231e-04,   4.61798423e-04],
       [  2.97529827e-04,   2.84079666e-04,   1.36522081e-04,
          3.14796271e-05,   2.04571952e-05,   5.83183146e-06],
       [  7.23517907e-04,   1.04410888e-03,   5.59139589e-04,
          1.72486514e-04,   3.23478052e-05,   2.28460485e-04],
       [  4.09871500e-05,   4.83260046e-05,   1.12100224e-05,
          3.66417771e-06,   3.85370527e-07,   4.48161984e-07],
       [  2.45718542e-03,   4.43953183e-03,   7.89466966e-03,
          1.96605106e-04,   4.63109900e-04,   7.69015576e-04],
       [  2.47963733e-04,   5.51276724e-04,   4.26518818e-04,
          1.62361903e-05,   3.42850944e-05,   1.36287839e-04],
       [  7.86206797e-02,   9.16792825e-02,   4.77645814e-01,
          8.95052124e-03,   7.97039829e-03,   1.10754184e-02],


In [15]:
cc = pd.concat([pd.DataFrame(df[i], columns=cols_target) for i in range(len(df))], ignore_index=True)

In [16]:
cc.head()

,obscene,insult,toxic,severe_toxic,identity_hate,threat
0,0.922573,0.586732,0.983838,0.002749,3.736567e-03,5.598809e-05
1,0.001451,0.003136,0.002781,0.000344,3.556172e-04,4.617984e-04
2,0.000298,0.000284,0.000137,0.000031,2.045720e-05,5.831831e-06
3,0.000724,0.001044,0.000559,0.000172,3.234781e-05,2.284605e-04
4,0.000041,0.000048,0.000011,0.000004,3.853705e-07,4.481620e-07


In [17]:
idlist = test_cleaned['id']
cc['id']=idlist
cc.head()

,obscene,insult,toxic,severe_toxic,identity_hate,threat,id
0,0.922573,0.586732,0.983838,0.002749,3.736567e-03,5.598809e-05,00001cee341fdb12
1,0.001451,0.003136,0.002781,0.000344,3.556172e-04,4.617984e-04,0000247867823ef7
2,0.000298,0.000284,0.000137,0.000031,2.045720e-05,5.831831e-06,00013b17ad220c46
3,0.000724,0.001044,0.000559,0.000172,3.234781e-05,2.284605e-04,00017563c3f7919a
4,0.000041,0.000048,0.000011,0.000004,3.853705e-07,4.481620e-07,00017695ad8997eb


In [18]:
cc = cc[['id','toxic','severe_toxic','obscene','threat','insult','identity_hate']]
cc.head()

,id,obscene,insult,toxic,severe_toxic,identity_hate,threat
0,00001cee341fdb12,0.922573,0.586732,0.983838,0.002749,3.736567e-03,5.598809e-05
1,0000247867823ef7,0.001451,0.003136,0.002781,0.000344,3.556172e-04,4.617984e-04
2,00013b17ad220c46,0.000298,0.000284,0.000137,0.000031,2.045720e-05,5.831831e-06
3,00017563c3f7919a,0.000724,0.001044,0.000559,0.000172,3.234781e-05,2.284605e-04
4,00017695ad8997eb,0.000041,0.000048,0.000011,0.000004,3.853705e-07,4.481620e-07


In [19]:
cc.to_csv('submission_1.csv', index=False)

In [84]:
result_7[0]

array([[  9.97106016e-01,   1.49514815e-02,   8.11845005e-01,
          9.64711711e-04,   6.41419530e-01,   3.33780306e-03,
          1.26189072e-04],
       [  1.36800038e-04,   5.29274912e-05,   1.65169826e-04,
          6.93944003e-06,   2.74036604e-04,   4.46183112e-05,
          9.99976397e-01],
       [  1.97102883e-04,   1.52482942e-04,   4.58621653e-04,
          6.32929587e-05,   5.51998091e-04,   6.32954398e-05,
          9.99960065e-01],
       [  5.42938185e-04,   5.18575776e-04,   3.77150252e-04,
          1.88391059e-04,   7.09902088e-04,   1.14672606e-04,
          9.99901295e-01],
       [  2.88672118e-06,   1.20481009e-05,   3.05976719e-05,
          1.21759498e-07,   2.73777496e-05,   1.12691475e-06,
          9.99999762e-01],
       [  5.92064857e-03,   4.73077816e-04,   1.71911356e-03,
          2.40918816e-04,   2.35621934e-03,   3.13433819e-04,
          9.97680187e-01],
       [  6.93830862e-06,   3.52015595e-05,   1.03421451e-04,
          3.31833508e-06,   2.08

In [87]:
import copy
df_7 = copy.deepcopy(result_7)

In [90]:
cc_7 = pd.concat([pd.DataFrame(df_7[i], columns=['toxic','severe_toxic','obscene','threat','insult','identity_hate','all_zero']) for i in range(len(df_7))], ignore_index=True)
cc_7.shape

(153164, 7)

In [93]:
cc_7=cc_7.drop('all_zero', axis=1)
cc_7.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.997106,0.014951,0.811845,9.647117e-04,0.641420,0.003338
1,0.000137,0.000053,0.000165,6.939440e-06,0.000274,0.000045
2,0.000197,0.000152,0.000459,6.329296e-05,0.000552,0.000063
3,0.000543,0.000519,0.000377,1.883911e-04,0.000710,0.000115
4,0.000003,0.000012,0.000031,1.217595e-07,0.000027,0.000001


In [95]:
idlist = test_cleaned['id']
cc_7['id']=idlist
cc_7 = cc_7[['id','toxic','severe_toxic','obscene','threat','insult','identity_hate']]
cc_7.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.997106,0.014951,0.811845,9.647117e-04,0.641420,0.003338
1,0000247867823ef7,0.000137,0.000053,0.000165,6.939440e-06,0.000274,0.000045
2,00013b17ad220c46,0.000197,0.000152,0.000459,6.329296e-05,0.000552,0.000063
3,00017563c3f7919a,0.000543,0.000519,0.000377,1.883911e-04,0.000710,0.000115
4,00017695ad8997eb,0.000003,0.000012,0.000031,1.217595e-07,0.000027,0.000001


In [96]:
cc_7.to_csv('submission_1_7.csv', index=False)